In [2]:
from itertools import product
from math import exp
import numpy as np
from numba import njit, prange

In [3]:
@njit
def int2array(x: int, L: int) -> np.ndarray:
    '''
    Transform number x to bit represantation with length of L
    '''
    res = np.empty(L, dtype=np.int8)
    for i in range(L):
        res[i] = (x & 1) * 2 - 1
        x = x >> 1
    return res

In [4]:
def get_range(a : int, b : int) -> np.array:
    '''
    get_range takes a and b and return range(a, b, 1). The difference is that this one return list.
    '''
    res = []
    while a <= b:
        res.append(a)
        a += 1
    return res

In [5]:

class find_mean_energy_for_1d:
    def int2array(self, x: int, L: int) -> np.ndarray:
        '''
        Transform integer x to bit represantation.
        '''
        res = np.empty(L, dtype=np.int8)
        for i in range(L):
            res[i] = (x & 1) * 2 - 1
            x = x >> 1
        return res
    def energy(self, sigma: np.ndarray) -> int:
        '''
        Return the energy of set of electrons when the spin orientation accords to sigma. It is working for 1D case.
        '''
        E = 0
        n = len(sigma)
        for i in range(n):
            E -= sigma[i] * sigma[(i + 1) % n]
        return E
    def mean_energy(self, L: int, kT: np.array) -> float:
        '''
        Return the mean energy of set of electrons with spin orientation accords to sigma and temperature equals to kT.
        '''
        E_mean = 0
        Z = 0
        for sigma in prange(2**(L-1)):
            E = self.energy(int2array(sigma, L))
            e = exp(-E / kT)
            E_mean += E * e
            Z += e
        E_mean /= Z
        return E_mean / L
    
@njit
def energy_for_2d(sigma) -> int:
    '''
    energy_for_2d calculates energy for given 2D matrix which contains information about spin orientation of electrons.
    '''
    E = 0
    n = len(sigma)
    m = len(sigma[0])
    for i in prange(-1, n - 1):
        for j in prange(-1, m - 1):
            E -= (sigma[i][j] * sigma[(i + 1)][j] + sigma[i][j] * sigma[i][j + 1])
    return E


In [6]:
@njit
def calculate_mean_energy_for_2d(sigma : np.array, Lx : int, E_mean : float, Z : float, kT : float) -> np.array:
    '''
    calculate_mean_energy_for_2d takes np.array numbers_for_translation_to_two_bits_base each number of which will be used to transfer bite representation
    to all spin sets then will use sigma to determine the mean energy E_mean. 
    '''
    E = energy_for_2d(sigma)
    e = np.exp(-E / kT)
    Z += e
    E_mean += E * e
    return np.array([E_mean, Z])

def mean_energy(Lx: int, Ly: int, kT: np.array) -> np.array:
    '''
    mean_energy function contains logic for finding mean energy for 1D case and 2D case. It depends on Lx and Ly which logic it will choose.
    '''
    i = 0
    if Lx == 1:
        res = np.empty(Ly)
        for current_kT in kT:
            res[i] = find_mean_energy_for_1d().mean_energy(Ly, current_kT)
            i += 1
        return res
    elif Ly == 1:
        res = np.empty(Lx)
        for current_kT in kT:
            res[i] = find_mean_energy_for_1d().mean_energy(Lx, current_kT)
            i += 1
        return res
    else:
        return mean_energy_for_2d(Lx, Ly, kT)
@njit(parallel=True)
def get_all_needed_sigmas(Number_of_rows_in_sigma : int, Number_of_columns_in_sigma : int) -> np.array:
    Number_of_elements_in_sigma = Number_of_rows_in_sigma * Number_of_columns_in_sigma
    res = np.empty((2**Number_of_elements_in_sigma, Number_of_rows_in_sigma, Number_of_columns_in_sigma), dtype=np.int8) # Each element of sigma can be 1 or -1 so 2 ** Number_of_elements_in_sigma give us all needed space
    for k in prange(2 ** Number_of_elements_in_sigma):
        sigma = np.empty((Number_of_rows_in_sigma, Number_of_columns_in_sigma), dtype=np.int8)
        current_number = np.int64(k)
        for index in range(Number_of_elements_in_sigma):
            if bool(current_number & 1): # If the last bit is 1
                sigma[index // Number_of_columns_in_sigma][index % Number_of_columns_in_sigma] = 1
            else:
                sigma[index // Number_of_columns_in_sigma][index % Number_of_columns_in_sigma] = -1
            current_number = current_number >> 1
        res[k] = sigma
        #yield sigma
    return res
                
@njit(parallel=True)
def mean_energy_for_2d(Lx: int, Ly : int, kT: np.array) -> np.array:
    '''
    mean_energy_for_2d calcultate mean energy of spin sets with Lx * Ly electrons. Lx number of rows and Ly number of columns in this matrix.
    '''

    Number_of_elements_in_sigma = Lx * Ly
    res = np.empty(kT.size, dtype=np.float32)
    #array_of_all_needed_sigmas = np.empty((2**Number_of_elements_in_sigma, Ly, Lx), dtype=np.int8) # On each row there will be sigma. Each sigma is an array with length of N.
    #array_of_all_needed_sigmas = get_all_needed_sigmas(Ly, Lx)

    for current_kT in prange(kT.size):
        E_mean = 0.0
        Z = 0.0
        for sigma in get_all_needed_sigmas(Ly, Lx):
            #Old logic
            # for row in sigma:
            #     row = np.append(row, 1)
            #New logic
            # new_column = np.array([1]*Ly)
            # new_column = new_column.reshape(sigma.shape[0], 1)
            # sigma = np.concatenate((sigma, new_column), axis=1)
            current_res = calculate_mean_energy_for_2d(sigma, Lx, E_mean, Z, kT[current_kT])
            E_mean = current_res[0]
            Z = current_res[1]
        E_mean /= Z
        res[current_kT] = E_mean / (Lx * Ly)        
    return res

In [ ]:
import time
from tqdm import trange, tqdm
# Initialization of data. Put your input here.
kT_range = np.arange(1.0, 5.0, 0.1) # Before multiplying by 0.1 and adding 1.0
Ly = 4
Lx_range = np.arange(2, 9, 1)

#Main logic
first_element_of_Lx_range = Lx_range[0]
#array_of_mean_energies = np.empty(shape=(Lx_range[-1] - Lx_range[0], int(kT_range[-1] * 10) - int(kT_range[0] * 10) + 1))
example = np.empty((1, 50))
for Lx in trange(2, 9):
    j = 0
    print(mean_energy_for_2d(Lx, Ly, kT_range))
    #array_of_mean_energies[Lx - first_element_of_Lx_range] = mean_energy_for_2d(Lx, Ly, kT_range)

#np.save("matrix_of_mean_energies.txt", array_of_mean_energies)

 43%|████████████████████████████████████                                                | 3/7 [00:04<00:04,  1.10s/it]

[-1.9950948  -1.9896786  -1.9807488  -1.9673111  -1.9485145  -1.9237583
 -1.8927729  -1.8556564  -1.8128631  -1.7651476  -1.7134756  -1.6589223
 -1.6025741  -1.5454491  -1.4884422  -1.4322953  -1.3775895  -1.3247523
 -1.2740754  -1.2257361  -1.1798202  -1.1363429  -1.0952675  -1.056521
 -1.0200056  -0.985609   -0.9532109  -0.9226886  -0.8939204  -0.86678827
 -0.84117913 -0.8169862  -0.7941093  -0.77245486 -0.75193626 -0.73247325
 -0.7139917  -0.69642335 -0.6797055  -0.6637802 ]
[-1.9971178  -1.9938366  -1.9882528  -1.9794935  -1.9665918  -1.9485321
 -1.9243262  -1.8931222  -1.8543358  -1.8077791  -1.7537512  -1.6930598
 -1.6269567  -1.5570004  -1.4848784  -1.4122306  -1.3405097  -1.2708944
 -1.2042588  -1.1411831  -1.0819927  -1.0268091  -0.9756014  -0.92823315
 -0.88450086 -0.84416294 -0.8069616  -0.7726371  -0.7409381  -0.7116267
 -0.6844822  -0.6593022  -0.6359026  -0.614117   -0.5937959  -0.5748049
 -0.5570237  -0.54034466 -0.5246712  -0.50991714]
[-1.9971584  -1.9939609  -1.988566

 57%|████████████████████████████████████████████████                                    | 4/7 [00:06<00:04,  1.44s/it]

[-1.9971589  -1.993963   -1.9885726  -1.9801838  -1.9678836  -1.9506396
 -1.9272969  -1.8966112  -1.8573564  -1.8085313  -1.7496502  -1.6810375
 -1.6039988  -1.5207558  -1.4341277  -1.3470633  -1.2621931  -1.1815336
 -1.1063887  -1.0374075  -0.97472644 -0.918132   -0.8672026  -0.82141644
 -0.78022444 -0.74309385 -0.70953214 -0.6790975  -0.65140074 -0.62610334
 -0.60291266 -0.5815769  -0.5618795  -0.54363453 -0.52668166 -0.51088274
 -0.4961181  -0.4822841  -0.4692903  -0.45705774]


 71%|████████████████████████████████████████████████████████████                        | 5/7 [00:46<00:28, 14.21s/it]

[-1.9971585  -1.9939611  -1.9885651  -1.9801579  -1.9678051  -1.9504296
 -1.926789   -1.8954941  -1.8551112  -1.8044033  -1.7427031  -1.6703254
 -1.5888273  -1.5009286  -1.410058   -1.3196926  -1.2327666  -1.1513411
 -1.0765613  -1.0088111  -0.9479399  -0.8934793  -0.84480935 -0.8012682
 -0.76221687 -0.72707146 -0.69531566 -0.6665018  -0.6402464  -0.6162225
 -0.59415233 -0.5738     -0.55496466 -0.5374748  -0.52118367 -0.5059651
 -0.4917101  -0.47832415 -0.46572503 -0.45384076]


In [3]:
array_of_mean_energies

NameError: name 'array_of_mean_energies' is not defined

In [9]:
data = np.load("matrix_of_mean_energies.txt")
psm = ax.pcolormesh(data, cmap=cmap, rasterized=True, vmin=-4, vmax=4)
ax

FileNotFoundError: [Errno 2] No such file or directory: 'matrix_of_mean_energies.txt'

In [12]:
@njit
def generator(n):
    for i in prange(n):
        yield i
for i in generator(400):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27